### *****Make sure to change the .csv file name in two places before running.*****
### Look for: # <-------------Change to: df_<Your File #>

In [7]:
#Import packages
import pandas as pd
import requests
import json
import numpy as np
import time
import gmaps

from config import (mqkey, gkey) 

In [ ]:
#load crash data
#file_path = 'Resources/CRASH DATA SET'

#crash_csv = pd.read_csv(file_path)

#place_holder_df = pd.DataFrame()

In [ ]:
#set list of lat lng to feed into google

#lat = df['Start_lat']
#lng = df['Start_lng']

#for row in crash_csv:
    #lat = df['Start_lat']
    #lng = df['Start_lng']
    #google_path = f'{lat},{lng}'
    #params = {'Key': gkey,
         #'Path' : google_path}
    #base_url = 'https://roads.googleapis.com/v1/speedLimits?'

    #response = requests.get(base_url, params=params)
    
    #roads_data = response.json()
    
    
    

In [ ]:
# convert to json
#roads_data = response.json()



In [ ]:
# Jon's code starts here.

### Make sure to change the .csv file name in two places before running.
### Look for: # <-------------Change to: df_<Your File #>

In [ ]:
# File to Load
file_to_load = "Resources/df_2.csv" # <-------------Change to: df_<Your File #>

# Read Purchasing File and store into Pandas data frame
df = pd.read_csv(file_to_load)

#print(df)

# Add "Speed Limit" and "Nearest Intersection" columns to the DataFrame.
df["Speed Limit"] = ""
df["Nearest Intersection"] = ""
#df

In [ ]:
# Set parameters for speed limit search.
params = {
    "key": mqkey,
    "includeRoadMetadata": "true",
    "includeNearestIntersection": "true"
}

# Set a variable to track each iteration.
Request_number = 0

# Use for loop to locate the speed limits.
for index, row in df.iterrows():
    
    # Get Lat & Lng from DataFrame.
    lat = row["Start_Lat"]
    lng = row["Start_Lng"]
    
    # Change location each iteration while leaving original params in place.
    #params["location"] = f'{lat},{lng}'
    location = f"{lat},{lng}"
    
    # Set base URL.
    base_url = "http://www.mapquestapi.com/geocoding/v1/reverse?includeRoadMetadata=true&includeNearestIntersection=true"
    query_url = f"{base_url}&key={mqkey}&location={location}"
    
    
    # Make requests and print URL.
    speed = requests.get(query_url)
    #print(speed.url)
    
    # Convert to json
    speed = speed.json()
    
    # In case there may be missing speed limits, use try-except to skip any missing data.
    try:
        df.loc[index, "Speed Limit"] = speed["results"][0]["locations"][0]["roadMetadata"]["speedLimit"]
        df.loc[index, "Nearest Intersection"] = speed["results"][0]["locations"][0]["nearestIntersection"]["streetDisplayName"]
        Request_number = Request_number + 1
        #time.sleep(.1)
        print(f"Processed request number {Request_number}")
        #if Request_number == 10:
            #break

    except Exception as e:
        Request_number = Request_number + 1
        print("Speed limit or Nearest Intersection not found... Next")
        print(e)
        #if Request_number == 10:
            #break
        
        pass  

In [ ]:
df

In [ ]:
# Count how many rows were blank in 'Speed Limit' column.
len(df[df['Speed Limit'] == ''])

In [ ]:
# Count how many rows were blank in 'Nearest Intersection' column.
len(df[df['Nearest Intersection'] == ''])

### Make sure to change the .csv file name in two places before running.
### Look for: # <-------------Change to: df_<Your File #>

In [ ]:
# Save to .csv
df.to_csv(r'Resources\df_2WithSpeed.csv', index=False) # <-------------Change to: df_<Your File #>

In [ ]:
#Jon's code ends here.

In [ ]:
#Calvin's Heatmap

In [8]:
# File to Load
file_to_load = "Resources/AccidentDataWithSpeed.csv" # <-------------Change to: df_<Your File #>

# Read Purchasing File and store into Pandas data frame
df = pd.read_csv(file_to_load)
df.head()

,ID,TMC,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Description,Number,Street,City,County,State,Temperature(F),Visibility(mi),Weather_Condition,Traffic_Signal,Speed Limit,Nearest Intersection
0,A-514241,201.0,2,2020-12-30 05:23:06,2020-12-30 07:28:47,44.861118,-93.176346,Right hand shoulder blocked due to accident on...,NaN,I-494 E,Saint Paul,Dakota,MN,23.0,10.0,Cloudy,False,60.0,NaN
1,A-514242,201.0,2,2020-12-30 05:44:12,2020-12-30 08:06:25,44.425419,-92.878181,Accident on US-52 Northbound after CR-1.,35598.0,Highway 52 Blvd,Cannon Falls,Goodhue,MN,26.0,10.0,Cloudy,False,65.0,360th Street Way
2,A-514243,201.0,2,2020-12-30 05:43:31,2020-12-30 07:09:40,44.807671,-93.398087,Accident on US-169 Northbound at Riverview Rd.,NaN,Shakopee Byp N,Minneapolis,Hennepin,MN,23.0,9.0,Cloudy,False,65.0,NaN
3,A-514244,201.0,2,2020-12-30 05:42:29,2020-12-30 07:36:15,44.946819,-93.079323,Right hand shoulder blocked due to accident on...,NaN,US-52 S,Saint Paul,Ramsey,MN,24.0,10.0,Cloudy,False,55.0,NaN
4,A-514245,201.0,2,2020-12-30 05:50:29,2020-12-30 08:06:13,44.268723,-94.725304,Accident on MN-4 near 200th St.,21221.0,State Highway 4,Sleepy Eye,Brown,MN,21.0,10.0,Cloudy / Windy,False,60.0,200th St


In [9]:
 # Configure gmaps with API key
gmaps.configure(api_key=gkey)

In [10]:
 # Store 'Lat' and 'Lng' into  locations 
locations = df[["Start_Lat", "Start_Lng"]].astype(float)

In [21]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=df["Severity"], 
                                 dissipating=False, max_intensity=4
                                 
                                 
                                 ,
                                 point_radius = .05)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))